# STEELTRACE™ LAB — PoC completo (Binder)

Pipeline reproducible del MVP académico:
1) MCP (ingesta + DQ) → 2) SHACL (E1/S1/G1) → 3) RAGA (KPIs + citas RAG) → 4) EEE-Gate → 5) XBRL → 6) Evidencias (Merkle) → 7) SLO p95 → 8) HITL κ → 9) ZIP

**Nota de rutas:** este cuaderno vive en `scripts/`, pero al iniciar cambiamos a la **raíz del repo** automáticamente.

In [ ]:
import os, pathlib
# Si el notebook está en /scripts, sube al root del repo
if pathlib.Path.cwd().name == 'scripts':
    os.chdir('..')
print('📁 Directorio de trabajo actual:', os.getcwd())
print('Contenido de nivel superior:', os.listdir('.'))

## 0) Comprobación rápida del entorno (opcional)
Si alguna dependencia faltara, revisa `requirements.txt` y relanza Binder.

In [ ]:
import sys
print('Python:', sys.version)
try:
    import pandas, rdflib, pyshacl, jsonschema, lxml
    print('Deps OK: pandas, rdflib, pyshacl, jsonschema, lxml')
except Exception as e:
    print('Aviso deps:', e)

## 1️⃣ MCP — Ingesta, validación de schema y DQ + linaje (hashes)

In [ ]:
!python scripts/mcp_ingest.py
print('\n✅ Ingesta/DQ completada. Normalizados:')
!ls -l data/normalized/
print('\n📄 DQ report:')
!sed -n '1,160p' data/dq_report.json
print('\n🔗 Linaje (primeras líneas):')
!sed -n '1,5p' data/lineage.jsonl

## 2️⃣ Ontología + SHACL — Validación E1/S1/G1 y grafo de linaje RDF (TTL)

In [ ]:
!python scripts/shacl_validate.py
from pathlib import Path
print('\n📄 ontology/validation.log (primeros 800 chars):')
print(Path('ontology/validation.log').read_text()[:800])
print('\n🔗 ontology/linaje.ttl (primeras 40 líneas):')
print('\n'.join(Path('ontology/linaje.ttl').read_text().splitlines()[:40]))

## 3️⃣ RAGA — KPIs E1/S1/G1 + explicaciones (hipótesis-evidencia) + citas RAG (índice versionado)

In [ ]:
!python scripts/raga_compute.py
from pathlib import Path
print('\n📈 KPIs:')
print(Path('raga/kpis.json').read_text())
print('\n🧩 Explicación:')
print(Path('raga/explain.json').read_text()[:1000])

## 4️⃣ EEE-Gate — scoring (Epistémico/Explícito/Evidencia) y decisión publish/review/block

In [ ]:
!python scripts/eee_gate.py
from pathlib import Path
print('\n🛡️ Gate report (primeros 800 chars):')
print(Path('ops/gate_report.json').read_text()[:800])
print('\n📄 eee_report.json:')
print(Path('eee/eee_report.json').read_text())

## 5️⃣ XBRL — Generación y validación contra XML Schema (Binder-friendly)
Para validación ESRS real, usar Arelle en entorno local (fuera de Binder) más adelante.

In [ ]:
!python scripts/xbrl_generate.py
from pathlib import Path
print('\n🧾 xbrl/informe.xbrl (primeras 20 líneas):')
print('\n'.join(Path('xbrl/informe.xbrl').read_text().splitlines()[:20]))
print('\n✅ xbrl/validation.log:')
print(Path('xbrl/validation.log').read_text())

## 6️⃣ Evidencias — Hash SHA256 + raíz Merkle + token TSA simulado + manifest JSON
Sustituye por TSA real (RFC3161) en despliegues fuera de Binder.

In [ ]:
!python scripts/evidence_build.py
from pathlib import Path
print('\n📚 Manifest (primeros 800 chars):')
print(Path('evidence/evidence_manifest.json').read_text()[:800])
print('\n🕒 Verificación TSA (simulada):')
print(Path('evidence/verify/2025Q1.txt').read_text())
print('\n📦 Tokens:')
!ls -l evidence/tokens/

## 7️⃣ SLO p95 — Orquestación cronometrada de todo el pipeline y acumulación histórica
En una ejecución el p95≈max; al repetir runs se estabiliza el p95. Reporte en `ops/slo_report.json` y `ops/slo_history.jsonl`.

In [ ]:
!python scripts/pipeline_run.py
from pathlib import Path
print('\n📊 SLO report:')
print(Path('ops/slo_report.json').read_text())
print('\n🗂️ History (últimas 5 líneas):')
!tail -n 5 ops/slo_history.jsonl || true

## 8️⃣ HITL — Acuerdo inter-evaluador (κ de Cohen) sobre DP críticos
Usa `docs/hitl_reviews.csv`. Objetivo orientativo κ≥0.70; si baja, afinar rúbrica/guía de revisión y muestreo de DPs.

In [ ]:
!python scripts/hitl_kappa.py
from pathlib import Path
print('\n🧪 κ results:')
print(Path('ops/hitl_kappa.json').read_text())

## 9️⃣ Paquete ZIP reproducible — Carpeta de evidencias y artefactos para la consultora
Genera un ZIP con todo lo necesario para replicar/verificar la corrida (release/audit/*.zip).

In [ ]:
!python scripts/package_release.py
!ls -l release/audit/ || true

## ✅ Resumen final de artefactos (OK/MISSING)

In [ ]:
from pathlib import Path
paths = [
  'data/normalized/energy_2024-01.json',
  'data/normalized/hr_2024-01.json',
  'data/normalized/ethics_2024-01.json',
  'ontology/validation.log', 'ontology/linaje.ttl',
  'raga/kpis.json', 'raga/explain.json',
  'ops/gate_report.json', 'eee/eee_report.json',
  'xbrl/informe.xbrl', 'xbrl/validation.log',
  'evidence/evidence_manifest.json', 'evidence/tokens/2025Q1.tsr',
  'ops/slo_report.json', 'ops/hitl_kappa.json'
]
print('📋 Resultado final:')
for p in paths:
    print(f'{p:55} →', '✅ OK' if Path(p).exists() else '⚠️ MISSING')

---
### Fin del flujo
- Si algo sale `MISSING`, vuelve a ejecutar la celda correspondiente.
- Para validación XBRL ESRS completa, usa Arelle localmente.
- Para TSA real (RFC3161), sustituye el módulo simulado en `evidence_build.py`.

Listo para revisión de la consultora y anexar a tu artículo como **material reproducible**.